In [3]:
import numpy as np
results_task1 = np.load('data/stateValuesTask1.npy')
grid_world = np.load('data/gridworld.npy')

In [18]:
goal = {'x':8,'y':3}

finished = False

def is_outside(state):
    return ((state['y'] < 0 or state['y'] >= grid_world.shape[0]) 
              or (state['x'] < 0 or state['x'] >= grid_world.shape[1]))

def terminal_state(state):
    # Entered a goal state
    if state == goal:
        return True
    # Exeeded boundary limits
    elif is_outside(state):
        return True
    # Entered a cell X
    elif grid_world[state['y'], state['x']] == -20:
        return True

    # Not a terminal state
    else:
        return False

def get_reward(state):
    if is_outside(state):
        return -30;
    else:
        return grid_world[state['y'], state['x']]

def make_move(state, action):
    state_c = state.copy()
    
    if action == 'l':
        state_c['x']  -= 1
    elif action == 'r':
        state_c['x']  += 1
    elif action == 'u':
        state_c['y']  -= 1
    elif action == 'd':
        state_c['y']  += 1
        
    if not terminal_state(state_c):
        state['x'] = state_c['x']
        state['y'] = state_c['y']
    
    return get_reward(state_c)

    
def compute_V(state, count, disc=0.9):
    if count == 7 or terminal_state(state):
        return 0
    
    return_V = 0
    count += 1

    # Right + random action
    state_r = state.copy()
    reward = make_move(state_r, 'r')
    return_V += (0.5 + 0.125) * (reward + disc**count * compute_V(state_r, count))
    
    # Random left action
    state_l = state.copy()
    reward = make_move(state_l, 'l')
    return_V += 0.125 * (reward + disc**count * compute_V(state_l, count))
    
    # Random up action
    state_u = state.copy()
    reward = make_move(state_u, 'u')
    return_V += 0.125 * (reward + disc**count * compute_V(state_u, count))
            
    # Random down action
    state_d = state.copy()
    reward = make_move(state_d, 'd')
    return_V += 0.125 * (reward + disc * compute_V(state_d, count))
        
    return return_V

In [19]:
state = {'x':8,'y':0}
compute_V(state, 0)

-69.79503857435238

In [20]:
np.set_printoptions(suppress=True)
grid_task1 = np.zeros_like(grid_world)

for y in range(grid_world.shape[0]):
    for x in range(grid_world.shape[1]):
        grid_task1[y,x] = compute_V({'y': y, 'x': x}, 0)
    
grid_task1

array([[ -4.95007053,  -4.9960163 , -15.53327745, -19.75023521,
        -22.82488322, -22.50716902, -24.68767579, -42.24101637,
        -69.79503857],
       [-16.05516738, -13.8662417 , -21.23297624, -33.13148269,
        -48.15431559,   0.        , -54.0605142 ,   0.        ,
        -69.6675506 ],
       [-54.75734238,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        , -66.73135202,   0.        ,
        -38.21512085],
       [-54.69750495,   0.        ,  -3.69401589, -10.94561142,
        -29.29937555, -55.52332406,   0.        , 174.63100811,
          0.        ],
       [-19.39244478, -14.66096936, -28.2945074 ,   0.        ,
          0.        ,   0.        , -15.35517201,  -3.43214664,
        -31.88796458],
       [-19.54246716, -16.27566294, -19.51451421, -26.16222685,
        -31.62396444, -39.22460656, -48.59047159,   0.        ,
        -74.44580755],
       [-43.17837835,   0.        ,   0.        ,   0.        ,
          0.        ,   0.    

In [17]:
results_task1

array([[ -55.72285158,  -60.28813092,  -76.21943677,  -85.94239006,
         -95.17214143, -102.06460192, -109.14849891, -128.03672486,
        -154.35820755],
       [ -70.24180495,  -71.03860383,  -82.4266548 ,  -97.04877034,
        -113.45253199,    0.        , -138.83622028,    0.        ,
        -140.1279616 ],
       [-114.35635123,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        , -156.41838188,    0.        ,
         -71.58053207],
       [-113.49056343,    0.        ,  -67.69447722,  -82.24055051,
        -104.22849427, -131.61383646,    0.        ,   67.97543479,
           0.        ],
       [ -66.87372941,  -61.70095357,  -77.43127146,    0.        ,
           0.        ,    0.        ,  -51.56857317,  -41.72612892,
         -63.11723718],
       [ -64.4840581 ,  -63.86563014,  -69.53147116,  -77.8922909 ,
         -83.19180027,  -88.82103118,  -95.51685587,    0.        ,
        -145.95964503],
       [ -81.41499169,    0.        ,   